In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd

In [34]:
#number 1 to 10 data
mnist=input_data.read_data_sets('MNIST_data',one_hot=True)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [35]:
def add_layer(inputs,in_size,out_size,activation_function=None):
    Weights=tf.Variable(tf.random_normal([in_size,out_size])) #tf.random_normal[row,col]
    biases=tf.Variable(tf.zeros([1,out_size])+0.1)
    Wx_plus_b=tf.matmul(inputs,Weights)+biases
    if activation_function is None:
        outputs=Wx_plus_b
    else:
        outputs=activation_function(Wx_plus_b)
    return outputs

In [36]:
def compute_accuracy(v_xs,v_ys):#validation_xs
    global prediction
    y_pre=sess.run(prediction,feed_dict={xs:v_xs})
    correct_prediction=tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    result=sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys})
    return result

In [37]:
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

In [38]:
def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [39]:
def conv2d(x,W):
    #stride[1,x_movement,y_movement,1]
    #must have strides[0]=strides[3]=1
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [40]:
def max_pool_2x2(x):
    #stride[1,x_movement,y_movement,1]    
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

最大池化這邊strides=[1,2,2,1]，一次跨兩步所以長寬縮小2倍

In [41]:
#define placeholder for inputs to network
xs=tf.placeholder(tf.float32,[None,784]) #28X28
ys=tf.placeholder(tf.float32,[None,10])
keep_prob=tf.placeholder(tf.float32)
x_image=tf.reshape(xs,[-1,28,28,1]) #[n_samples,28,28,1] :-1會自動算出n個28*28*1的example


In [42]:
##conv1 layer##
#作1次捲積與1次最大池化
W_conv1=weight_variable([5,5,1,32]) #filter 5X5,in size 1 ,out size 32
b_conv1=bias_variable([32])
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)#output size 28X28X32
h_pool1=max_pool_2x2(h_conv1)                      #output size 14X14X32

灰階圖片只有一個channel所以input size=1，如果是彩色圖片就有RGB 3個channel，input=3

In [43]:
##conv2 layer##
W_conv2=weight_variable([5,5,32,64]) #filter 5X5,in size 32 ,out size 64
b_conv2=bias_variable([64])
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)#output size 14X14X64
h_pool2=max_pool_2x2(h_conv2)                      #output size 7X7X64

In [44]:
###func1 layer###
W_fc1=weight_variable([7*7*64,1024])
b_fc1=bias_variable([1024])
#[n_samples,7,7,64]變成 [n_samples,7*7*64]
h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64])
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob) #避免overfitting

###func2 layer###
W_fc2=weight_variable([1024,10])
b_fc2=bias_variable([10])


prediction=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)




In [45]:
#add output layer
prediction=add_layer(xs,784,10,activation_function=tf.nn.softmax)

In [46]:
cross_entropy=tf.reduce_mean(-tf.reduce_sum(ys*tf.log(prediction),reduction_indices=[1]))
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

In [47]:
sess=tf.Session()
sess.run(tf.initialize_all_variables())

In [48]:
for i in range(1000):
    batch_xs,batch_ys=mnist.train.next_batch(100)
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys})
    if i % 50==0:
        print(compute_accuracy(mnist.test.images,mnist.test.labels))
        

0.1355
0.1382
0.1394
0.1419
0.1452
0.1487
0.1513
0.1537
0.1562
0.1596
0.1623
0.1667
0.1712
0.1754
0.1803
0.1844
0.1903
0.197
0.2019
0.2091
